In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Setup

In [11]:
x_train = np.load("../input/ndscloaded/x_train.npy")
x_test = np.load("../input/ndscloaded/x_test.npy")
y_train = np.load("../input/ndscloaded/y_train.npy")
features = np.load("../input/ndscloaded/features.npy")
test_features = np.load("../input/ndscloaded/test_features.npy")
word_index = np.load("../input/ndscloaded/word_index.npy").item()
embedding_matrix = np.load("../input/ndscloaded/embedding_matrix.npy")

In [ ]:
print(y_train)

In [ ]:
print(np.shape(embedding_matrix[25]))

In [ ]:
batch_size = 128

indices = np.arange(len(x_train)-1)
np.random.shuffle(indices)
i = 0
x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
y_train_batch = np.zeros((batch_size, 58))
y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

print(indices)
print(x_train_batch_tokens)
print(np.shape(x_train_batch[:1]))
print(y_train_batch_targets)
print(y_train_batch)

In [12]:
# Data providers
batch_size = 128

indices = np.arange(len(x_train)-1)
# print(len(x_train))
# print(np.random.shuffle())

def batch_gen():
    n_batches = math.ceil(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
#         print(indices)
        for i in range(n_batches):
#             texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
#             text_arr = np.array([text_to_array(text) for text in texts])
            
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
            y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
            y_train_batch = np.zeros((batch_size, 58))
            y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

            yield (x_train_batch, y_train_batch)

# Training

In [13]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation

In [14]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(30, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(58))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
mg = batch_gen()
model.fit_generator(mg, epochs=1,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/1


AttributeError: 'list' object has no attribute 'shape'

# Inference

In [ ]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = pd.read_csv("../input/ndsc-beginner/test.csv")

all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

In [ ]:
print(np.shape(all_preds)
y_te = [np.argmax(pred) for pred in all_preds]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)